In [ ]:
# Preprocessing of the AbdomenCT-1K dataset. This includes
# - Resampling to a voxel size of 2x2x3mm
# - Registration to sample <Case_00001>

# Preliminaries: Download data via link in https://github.com/JunMa11/AbdomenCT-1K and copy the images and labels into
# a directory per subject, e.g., via 
# for i in `cat sublist.txt`; do mv "AbdomenCT-1K-ImagePart1/${i}_0000.nii.gz" "${i}/img.nii.gz"; done

# Author: Fabian Bongratz, 25.1.23

In [ ]:
PATH = '/path/to/AbdomenCT-1K'
ID_FILE = '/path/to/sublist.txt'

# Scans with missing organs will be ignored
MISSING_ORGAN_FILE = '/path/to/missing_organ_ids.txt'

ids = [line.rstrip('\n') for line in open(ID_FILE, 'r')]
print(f"{len(ids)} IDs in total")

In [ ]:
def show_slices(img):
    shape = img.shape
    fig, axs = plt.subplots(1,3)
    axs[0].imshow(np.rot90(img[:, :, shape[2]//2]), cmap="gray")
    axs[1].imshow(np.flip(np.rot90(img[:, shape[1]//2, :], k=3), axis=0), cmap="gray")
    axs[2].imshow(np.flip(np.rot90(img[shape[0]//2, :, :], k=3), axis=0), cmap="gray")
    plt.tight_layout()

In [ ]:
import nibabel as nib
import numpy as np
import os
import matplotlib.pyplot as plt
img = nib.load("Case_00001/img.nii.gz")
img_arr = img.get_fdata()
img_affine = img.affine
print(f"Affine:\n{img_affine}")
shape = img_arr.shape
print(f"Image shape: {shape}")
show_slices(img_arr)

In [ ]:
cnt, bins = np.histogram(img_arr)
_ = plt.hist(img_arr.flatten(), bins=100)

In [ ]:
import SimpleITK as sitk
SPACING = [2.0, 2.0, 3.0]

In [ ]:
def resample_volume(volume_path, new_spacing, interpolator=sitk.sitkLinear):
    """From https://discourse.itk.org/t/resample-volume-to-specific-voxel-spacing-simpleitk/3531"""
    #dtype = sitk.sitkInt32 if interpolator == sitk.sitkNearestNeighbor else sitk.sitkFloat32
    volume = sitk.ReadImage(volume_path)
    original_spacing = volume.GetSpacing()
    print(f"Original spacing {original_spacing}")
    print(f"New spacing {new_spacing}")
    original_size = volume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    return sitk.Resample(
        volume, new_size, sitk.Transform(), interpolator,
        volume.GetOrigin(), new_spacing, volume.GetDirection(), 0,
        volume.GetPixelID()
    )

In [ ]:
# Resample volumes

writer = sitk.ImageFileWriter()

for i in ids:
    print(f"### Processing ID {i} ###")
    img_file = os.path.join(PATH, i, "img.nii.gz")
    label_file = os.path.join(PATH, i, "label.nii.gz")
    # Resample as done in https://kits.lib.umn.edu/an-attempt-at-beating-the-3d-u-net/
    resampled_img = resample_volume(img_file, SPACING, interpolator=sitk.sitkBSplineResamplerOrder3)
    resampled_label = resample_volume(label_file, SPACING, interpolator=sitk.sitkNearestNeighbor)
    
    out_img_file = os.path.join(PATH, i, "resampled_img.nii.gz")
    writer.SetFileName(out_img_file)
    writer.Execute(resampled_img)
    
    out_label_file = os.path.join(PATH, i, "resampled_label.nii.gz")
    writer.SetFileName(out_label_file)
    writer.Execute(resampled_label)

In [ ]:
img = nib.load("Case_00001/resampled_img.nii.gz")
img_arr = img.get_fdata()
img_affine = img.affine
print(f"Affine:\n{img_affine}")
shape = img_arr.shape
print(f"Image shape: {shape}")
show_slices(img_arr)

In [ ]:
# Register volumes
import subprocess
REF_FILE = os.path.join(PATH, "Case_00001/resampled_img.nii.gz")
for i in ids:
    print(f"### Processing ID {i} ###")
    img_file = os.path.join(PATH, i, "resampled_img.nii.gz")
    label_file = os.path.join(PATH, i, "resampled_label.nii.gz")
    
    out_img_file = os.path.join(PATH, i, "registered_img.nii.gz")
    out_label_file = os.path.join(PATH, i, "registered_label.nii.gz")
    out_affine_file = os.path.join(PATH, i, "affine.txt")
    
    # niftyreg registration     
    subprocess.call(['reg_aladin', '-ref', REF_FILE, '-flo', img_file, '-aff', out_affine_file, '-voff'])
    if os.path.isfile('outputResult.nii.gz'):
        os.remove('outputResult.nii.gz')
    subprocess.call(['reg_resample', '-ref', REF_FILE, '-flo', img_file, '-trans', out_affine_file, '-res', out_img_file, '-inter', '3', '-pad', '-1024', '-voff'])
    subprocess.call(['reg_resample', '-ref', REF_FILE, '-flo', label_file, '-trans', out_affine_file, '-res', out_label_file, '-inter', '0', '-voff'])

In [ ]:
img = nib.load("Case_00002/resampled_img.nii.gz")
img_arr = img.get_fdata()
img_affine = img.affine
print(f"Affine:\n{img_affine}")
shape = img_arr.shape
print(f"Image shape: {shape}")
show_slices(img_arr)

In [ ]:
img = nib.load("Case_00002/registered_img.nii.gz")
img_arr = img.get_fdata()
img_affine = img.affine
print(f"Affine:\n{img_affine}")
shape = img_arr.shape
print(f"Image shape: {shape}")
show_slices(img_arr)

In [ ]:
def create_mesh_from_voxels(volume, mc_step_size=1):
    """ Convert a voxel volume to mesh using marching cubes

    :param volume: The voxel volume.
    :param mc_step_size: The step size for the marching cubes algorithm.
    :return: The generated mesh.
    """
    # Pad volume to avoid holes in the surfaces on the volume boundary
    volume = np.pad(volume, 1)
    
    vertices_mc, faces_mc, _, _ = measure.marching_cubes(
        volume, 0, step_size=mc_step_size, allow_degenerate=False
    )

    # measure.marching_cubes uses left-hand rule for normal directions, our
    # convention is right-hand rule
    faces_mc = np.flip(faces_mc, axis=[1])
    
    # We added one row of voxels during padding
    vertices_mc = vertices_mc - 1

    return trimesh.Trimesh(vertices_mc, faces_mc, process=False)

def mc_and_store(voxel_label, affine, filename):
    # Marching cubes
    mesh = create_mesh_from_voxels(voxel_label)
    # Voxel --> world coordinates
    mesh.apply_transform(affine)
    mesh.export(filename)

In [ ]:
# Extract meshes
import trimesh
from scipy import ndimage
from skimage import measure

label_names = {"background": 0, "liver": 1, "kidney": 2, "spleen": 3, "pancreas": 4}

missing_organ_ids = []

for i in ids:
    print(f"### Processing ID {i} ###")
    label_file = os.path.join(PATH, i, "registered_label.nii.gz")
    label = nib.load(label_file)
    vox2world = label.affine
    label_arr = label.get_fdata()
    
    for label_name, label_id in label_names.items():
        # Ignore background
        if label_name == "background":
            continue
        label_binary = (label_arr == label_id).astype(np.int32)
        # Split kidneys
        if label_name == "kidney":
            # Opening/closing
            new_binary = ndimage.binary_opening(
                ndimage.binary_closing(
                    label_binary,
                    structure=np.ones((5,5,5)),
                ),
                structure=np.ones((5,5,5))
            )
            split_map, n = ndimage.label(new_binary)
            if n > 2:
                print("More than two kidney labels, please check morphology operations.")
                continue

            x1, _, _ = np.where(split_map == 1)
            x2, _, _ = np.where(split_map == 2)

            # Find left and right kidney according to their x coordinate
            if np.average(x1) > split_map.shape[0] / 2:
                right_kidney_map = (split_map == 1).astype(np.int32)
                left_kidney_map = (split_map == 2).astype(np.int32)
            else:
                right_kidney_map = (split_map == 2).astype(np.int32)
                left_kidney_map = (split_map == 1).astype(np.int32)

            right_out_fn_full = os.path.join(
                PATH, i, "registered_" + label_name + "_right.ply"
            )
            try:
                mc_and_store(right_kidney_map, vox2world, right_out_fn_full)
            except RuntimeError:
                missing_organ_ids.append(i)
                print(f"Skipping file {right_out_fn_full}")
            left_out_fn_full = os.path.join(
                PATH, i, "registered_" + label_name + "_left.ply"
            )
            try:
                mc_and_store(left_kidney_map, vox2world, left_out_fn_full)
            except RuntimeError:
                missing_organ_ids.append(i)
                print(f"Skipping file {left_out_fn_full}")

            continue

        # All other organs
        out_fn_full = os.path.join(
                PATH, i, "registered_" + label_name + ".ply"
        )
        try:
            mc_and_store(label_binary, vox2world, out_fn_full)
        except RuntimeError:
            missing_organ_ids.append(i)
            print(f"Skipping file {out_fn_full}")
            
with open(MISSING_ORGAN_FILE, 'w') as file:
    for i in missing_organ_ids:
        file.write(str(i) + '\n')